In [1]:
from data.energy_service import EnergyService

from pandas import DataFrame
from model.series_sample import TimeSeriesSampleAccessor
# Accessor does not work unless working with a specific kernal? TODO: Managing kernals
# https://medium.com/@eleroy/jupyter-notebook-in-a-virtual-environment-virtualenv-8f3c3448247
from data.schemas.influx_schema import DailyReadingSchema, HHourlyReadingSchema, SolarControlSchema, SolarPandOSchema

import json
import os

In [2]:
INFLUX_TS_FMT = "%Y-%m-%dT%H:%M:%SZ"

In [3]:
service = EnergyService()
res = service.get_readings("daily", "MAC000246", "2011-04-12 10:30:00.0000000", "2012-04-12 10:30:00.0000000")

In [4]:
table, timestamps, lables, data = \
    service.parse_response(res, DailyReadingSchema())

In [5]:
sample = DataFrame(data=data, columns=lables)
sample['timestamp'] = timestamps

sample.tss.format_index(timestamps, INFLUX_TS_FMT)

In [6]:
sample.describe()

,energy_count,energy_max,energy_mean,energy_median,energy_min,energy_std,energy_sum
count,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000
mean,47.863636,1.209492,0.247898,0.111034,0.038318,0.293718,11.858182
std,1.566699,0.277129,0.092312,0.064607,0.012987,0.083685,4.432556
min,30.000000,0.218000,0.082042,0.063500,0.017000,0.026271,3.938000
25%,48.000000,1.056750,0.186927,0.078375,0.028000,0.251286,8.972500
50%,48.000000,1.194500,0.222875,0.094250,0.036000,0.286858,10.588000
75%,48.000000,1.339500,0.300633,0.114250,0.044250,0.342004,14.398750
max,48.000000,1.981000,0.609187,0.572500,0.074000,0.484152,29.241000


In [7]:
sample.head()

,energy_count,energy_max,energy_mean,energy_median,energy_min,energy_std,energy_sum,mac_id,timestamp
2011-12-03,30,1.179,0.300233,0.1150,0.047,0.361231,9.007,MAC000246,2011-12-03T00:00:00Z
2011-12-04,48,1.914,0.432563,0.1750,0.026,0.484152,20.763,MAC000246,2011-12-04T00:00:00Z
2011-12-05,48,0.934,0.125417,0.0780,0.025,0.153327,6.020,MAC000246,2011-12-05T00:00:00Z
2011-12-06,48,1.216,0.277542,0.1140,0.027,0.313399,13.322,MAC000246,2011-12-06T00:00:00Z
2011-12-07,48,0.873,0.188792,0.1035,0.026,0.209613,9.062,MAC000246,2011-12-07T00:00:00Z


In [10]:
sample.tss.stationality('energy_mean')

Stationality of energy_mean


{'adf': -1.7509470692868254,
 'pvalue': 0.4050724062676825,
 'usedlag': 13,
 'nobs': 118,
 'values': {'1%': -3.4870216863700767,
  '5%': -2.8863625166643136,
  '10%': -2.580009026141913},
 'icbest': -244.8412703786973}

In [11]:
sample.tss.autocorrelation('energy_mean')

array([ 1.        ,  0.01988124, -0.07587133, -0.0857847 , -0.1294209 ,
       -0.16521564,  0.042743  ,  0.17898557,  0.08994291, -0.12362803,
       -0.14154587, -0.0384344 , -0.02130338,  0.03016252,  0.57049458,
        0.07028073, -0.0676244 , -0.1082226 , -0.19143979, -0.15864797,
        0.10618308,  0.19751814,  0.22004059, -0.12659641, -0.09526823,
       -0.09804749, -0.03836123, -0.01841888,  0.51543392, -0.02513004,
       -0.03867825, -0.1352832 , -0.18081904, -0.15329149,  0.00726753,
        0.28549599,  0.10829312, -0.10260334, -0.17825569, -0.08851259,
       -0.15899075, -0.0245425 ,  0.557379  ,  0.03499979, -0.04936955,
       -0.14580598, -0.21307712, -0.10908383, -0.00558433,  0.19076638,
        0.06413847, -0.11978583, -0.13446265, -0.02350572, -0.17832074,
       -0.02620031,  0.5161205 , -0.01199376, -0.15238546, -0.10178701,
       -0.17073283, -0.01778951, -0.02984613,  0.1285371 ,  0.05905938,
       -0.10444507, -0.19666894, -0.01203473, -0.18859863,  0.00